In [8]:
import cv2
import numpy as np

In [2]:
image = cv2.imread("D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\bmp\\lena.bmp")

In [49]:
block_8x8 = image[:8,:8,0].copy()
bits = np.unpackbits(block_8x8)
array_bits = bits.reshape((8,8,8))
array_bits[:,:,-2:] = 0

In [17]:
new_block = np.packbits(array_bits).reshape((8,8))

In [16]:
print(new_block.shape)

(64,)


In [18]:
print(block_8x8)

[[121 121 129 121 121 113 121 121]
 [121 121 129 121 121 113 121 121]
 [121 121 129 121 121 113 121 121]
 [121 121 129 121 121 113 121 121]
 [121 121 129 121 121 113 121 121]
 [121 121 105 109 121 113 121 105]
 [121 121 109 113 121 105 113 101]
 [121 121 101 109 121 113 109 105]]


In [19]:
print(new_block)

[[120 120 128 120 120 112 120 120]
 [120 120 128 120 120 112 120 120]
 [120 120 128 120 120 112 120 120]
 [120 120 128 120 120 112 120 120]
 [120 120 128 120 120 112 120 120]
 [120 120 104 108 120 112 120 104]
 [120 120 108 112 120 104 112 100]
 [120 120 100 108 120 112 108 104]]


In [40]:
c = np.array(cv2.dct(np.array(new_block,dtype = np.float32)),dtype=np.int32)

In [60]:
print(c)

[[937  17   0  -1   2   0 -11  14]
 [ 26  -5   0 -23  -2   6  12   4]
 [-11   3   0   9   2  -4  -3  -3]
 [ -1   0   0   2  -2   2  -3   1]
 [  6  -1   0  -5   1  -1   5  -1]
 [ -2   3  -1   0   0   1  -2   1]
 [ -2  -3   2   7  -1  -1  -1  -1]
 [  3   2  -1  -6   0   1   2   1]]


In [59]:
print(int(format(c[2,0],'b'),2))

-11


In [61]:
test_bits = format(c[2,0],'b')

In [62]:
test_bits

'-1011'

In [65]:
new_test_bits = test_bits[:-1] + '0'

In [66]:
print(int(new_test_bits,2))

-10


In [41]:
i_block = cv2.idct(np.array(c,dtype=np.float32))

In [42]:
print(np.array(i_block,dtype=np.uint8))

[[120 120 128 120 119 112 119 119]
 [119 119 127 120 119 111 119 120]
 [120 119 127 119 119 112 120 119]
 [120 119 127 120 120 111 119 119]
 [119 120 127 120 119 111 119 118]
 [119 119 105 109 120 111 119 104]
 [119 120 107 111 119 105 112 100]
 [119 119 101 108 120 111 109 103]]


In [37]:
mse = np.mean((np.array(np.array(i_block,dtype=np.uint8),dtype = np.float32) - np.array(block_8x8,dtype=np.float32))**2)

In [39]:
print(cv2.PSNR(block_8x8,np.array(i_block,dtype=np.uint8)))

49.66047821076454


2,0 1,1 0,2 0,3

In [43]:
print(c)

[[937  17   0  -1   2   0 -11  14]
 [ 26  -5   0 -23  -2   6  12   4]
 [-11   3   0   9   2  -4  -3  -3]
 [ -1   0   0   2  -2   2  -3   1]
 [  6  -1   0  -5   1  -1   5  -1]
 [ -2   3  -1   0   0   1  -2   1]
 [ -2  -3   2   7  -1  -1  -1  -1]
 [  3   2  -1  -6   0   1   2   1]]


In [78]:
def embed_onebit(c,bit):
    bits =  format(c,'b')
    new_bits = bits[:-1] + format(bit,'b')
    return int(bits,2)

In [79]:
def embed_4bits(block_8x8,bits):
    block_bits = np.unpackbits(block_8x8)
    array_bits = block_bits.reshape((8,8,8))
    # array_bits[:,:,-2:] = 0
    new_block = np.packbits(array_bits).reshape((8,8))
    coeff = np.array(cv2.dct(np.array(new_block,dtype = np.float32)),dtype=np.int32)

    list_loc = [[2,0],[1,1],[0,2],[0,3]]
    for i in range(4):
        r,c = list_loc[i]
        coeff[r,c] = embed_onebit(coeff[r,c],bit=bits[i])
    i_block = cv2.idct(np.array(coeff,dtype=np.float32))
    return np.array(i_block,dtype=np.uint8)

In [80]:
def extract_4bits(w_block_8x8):
    block_bits = np.unpackbits(w_block_8x8)
    array_bits = block_bits.reshape((8,8,8))
    # array_bits[:,:,-2:] = 0
    new_block = np.packbits(array_bits).reshape((8,8))
    coeff = np.array(cv2.dct(np.array(new_block,dtype = np.float32)),dtype=np.int32)
    list_loc = [[2,0],[1,1],[0,2],[0,3]]
    w_bits = []
    for i in range(4):
        r,c = list_loc[i]
        bits =  format(coeff[r,c],'b')
        w_bits.append(bits[-1])
    return w_bits

In [81]:
list_bits = []
for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                list_bits.append([i,j,k,l])

In [75]:
print(len(list_bits))

16


In [87]:
out_list = []
for bits in list_bits[1:2]:
    w_block = embed_4bits(block_8x8=block_8x8,bits=bits)
    out_list.append(extract_4bits(w_block_8x8=w_block))

    

In [88]:
out_list

[['0', '0', '0', '1']]